In [13]:
import sqlite3
import pandas as pd
import numpy as np
from datetime import timedelta

In [11]:
connection = sqlite3.connect("../data/fpl.db")
raw_data = pd.read_sql_query("""SELECT season, player, pos, round, min, date
                             FROM raw_player_match_log 
                             where comp='Premier League'
                             and player = 'Bukayo Saka'""", connection)
raw_data["date"] = pd.to_datetime(raw_data["date"])
raw_data.head()

,season,player,pos,round,min,date
0,2021-2022,Bukayo Saka,LW,Matchweek 1,32,2021-08-13
1,2021-2022,Bukayo Saka,LW,Matchweek 2,60,2021-08-22
2,2021-2022,Bukayo Saka,RM,Matchweek 3,45,2021-08-28
3,2021-2022,Bukayo Saka,LW,Matchweek 4,90,2021-09-11
4,2021-2022,Bukayo Saka,"LM,LW",Matchweek 5,90,2021-09-18


In [43]:
def create_ma_feature(df: pd.DataFrame, match_stat_col: str, lag: int):
    df = df.sort_values(by=["player", "date"])

    # Create moving average
    ma_df = (
        df.groupby("player")
        .apply(lambda x: calculate_multi_lag_ma(x, match_stat_col, lag))
    )
    display(df)
    display(ma_df)
    return pd.merge(df, ma_df, left_index=True, right_index=True)


from pandas.core.groupby.generic import DataFrameGroupBy
def calculate_multi_lag_ma(group: DataFrameGroupBy, match_stat_col: str, max_lag: int):
    ma_df = pd.DataFrame(index=group.index)
    
    for i in range(len(group)):
        for lag in range(1, max_lag+1):            
            if i >= lag and group["date"].iloc[i] - group["date"].iloc[i - lag] <= timedelta(days=365):
                ma_df.loc[group.index[i], f"{match_stat_col}_ma{lag}"] = group[match_stat_col].iloc[i - lag : i].mean()
            
    return ma_df

df = raw_data
result = create_ma_feature(df, "min", 5)
result

C:\Users\ongmi\AppData\Local\Temp\ipykernel_23180\233912642.py:7: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(lambda x: calculate_multi_lag_ma(x, match_stat_col, lag))


,season,player,pos,round,min,date
96,2018-2019,Bukayo Saka,LW,Matchweek 21,8,2019-01-01
70,2019-2020,Bukayo Saka,LW,Matchweek 6,45,2019-09-22
71,2019-2020,Bukayo Saka,LW,Matchweek 7,79,2019-09-30
72,2019-2020,Bukayo Saka,"LW,RW",Matchweek 8,82,2019-10-06
73,2019-2020,Bukayo Saka,"LW,AM",Matchweek 9,90,2019-10-21
...,...,...,...,...,...,...
130,2022-2023,Bukayo Saka,RW,Matchweek 34,73,2023-05-02
131,2022-2023,Bukayo Saka,RW,Matchweek 35,86,2023-05-07
132,2022-2023,Bukayo Saka,RW,Matchweek 36,90,2023-05-14
133,2022-2023,Bukayo Saka,RW,Matchweek 37,90,2023-05-20


,min_ma1,min_ma2,min_ma3,min_ma4,min_ma5
96,NaN,NaN,NaN,NaN,NaN
70,8.0,NaN,NaN,NaN,NaN
71,45.0,26.5,NaN,NaN,NaN
72,79.0,62.0,44.000000,NaN,NaN
73,82.0,80.5,68.666667,53.50,NaN
...,...,...,...,...,...
130,79.0,84.5,86.333333,87.25,76.0
131,73.0,76.0,80.666667,83.00,84.4
132,86.0,79.5,79.333333,82.00,83.6
133,90.0,88.0,83.000000,82.00,83.6


,season,player,pos,round,min,date,min_ma1,min_ma2,min_ma3,min_ma4,min_ma5
96,2018-2019,Bukayo Saka,LW,Matchweek 21,8,2019-01-01,NaN,NaN,NaN,NaN,NaN
70,2019-2020,Bukayo Saka,LW,Matchweek 6,45,2019-09-22,8.0,NaN,NaN,NaN,NaN
71,2019-2020,Bukayo Saka,LW,Matchweek 7,79,2019-09-30,45.0,26.5,NaN,NaN,NaN
72,2019-2020,Bukayo Saka,"LW,RW",Matchweek 8,82,2019-10-06,79.0,62.0,44.000000,NaN,NaN
73,2019-2020,Bukayo Saka,"LW,AM",Matchweek 9,90,2019-10-21,82.0,80.5,68.666667,53.50,NaN
...,...,...,...,...,...,...,...,...,...,...,...
130,2022-2023,Bukayo Saka,RW,Matchweek 34,73,2023-05-02,79.0,84.5,86.333333,87.25,76.0
131,2022-2023,Bukayo Saka,RW,Matchweek 35,86,2023-05-07,73.0,76.0,80.666667,83.00,84.4
132,2022-2023,Bukayo Saka,RW,Matchweek 36,90,2023-05-14,86.0,79.5,79.333333,82.00,83.6
133,2022-2023,Bukayo Saka,RW,Matchweek 37,90,2023-05-20,90.0,88.0,83.000000,82.00,83.6
